In [ ]:
import sys
import nanohubremote as nr
import nanohubuidl
import warnings
warnings.filterwarnings("ignore")
import os
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"] 

In [ ]:
tool = nr.Sim2l(auth_data)

In [ ]:
SIMTOOLNAME = "caecipher"
schema = tool.getSchema(SIMTOOLNAME)
params = tool.getToolParameters(SIMTOOLNAME)

## Importing libraries

In [ ]:
import sys
import json
import nanohubuidl.teleport as t
from nanohubuidl.simtool import SimtoolBuilder
from nanohubuidl.material import MaterialBuilder
from nanohubuidl.material import MaterialContent, MaterialLabContent
from nanohubuidl.plotly import PlotlyBuilder
from nanohubuidl.app import AppBuilder

from nanohubuidl.nanohub import Auth
TOOLNAME = schema["name"]# "st4pnjunction"
REVISION = schema["revision"]# 6
STATE_LOADER_STATUS = "loader_status"
STATE_LOADER_OPEN = "loader_open"
STATE_LOGIN_OPEN = "login_open"
STATE_ERROR_STATUS = "error_status"
STATE_ERROR_OPEN = "error_open"
STATE_ALERT_STATUS = "alert_status"
STATE_ALERT_OPEN = "alert_open"

## Creating the project and adding GUI variables 

In [ ]:
Project = t.TeleportProject("PNToy Lab")
Component = Project.root


In [ ]:
Project.libraries['react'] = 'https://nanohub.org/js_apps/react.production.min'
Project.libraries['react-dom'] = 'https://nanohub.org/js_apps/react-dom.production.min'
Project.libraries['material-ui'] = 'https://nanohub.org/js_apps/material-ui.production.min'
Project.libraries['materiallab-ui'] = 'https://nanohub.org/js_apps/material-ui-lab.production.min'
Project.libraries['math'] = 'https://cdnjs.cloudflare.com/ajax/libs/mathjs/6.6.1/math.min'
Project.libraries['axios'] = 'https://nanohub.org/js_apps/axios.min'
Project.libraries['localforage'] = 'https://nanohub.org/js_apps/localforage.min'
Project.libraries['prop-types'] = 'https://nanohub.org/js_apps/prop-types.min'

## Creating Error

In [ ]:
ErrorMessage = SimtoolBuilder.Error( 
    Component, 
    error_status = STATE_ERROR_STATUS,
    error_open = STATE_ERROR_OPEN,
    is_open = False
)

## Creating ALERT Mesage

In [ ]:
AlertMessage = SimtoolBuilder.Error( 
    Component, 
    error_status = STATE_ALERT_STATUS,
    error_open = STATE_ALERT_OPEN,
    is_open = False,
    title = "Message"
)
AlertMessage.content.children[0].content.style['backgroundColor'] = '#FFF380'

## Creating loader

In [ ]:
Loader = SimtoolBuilder.Loader( 
    Component, 
    loader_status = STATE_LOADER_STATUS,
    loader_open = STATE_LOADER_OPEN,
    is_open = False
)

## Schema

In [ ]:
SimtoolBuilder.buildSchema(
    Project,
    Component,
    url = "https://nanohub.org/api/dbexplorer/simtools",
    toolname = TOOLNAME,
    revision = REVISION
)
Component.addPropVariable(
    "onLoadSchema", 
    {
        "type" : "func", 
        'defaultValue' : 
        '(e)=>{e.setState({"'+ STATE_LOADER_OPEN +'":false})}'
    }
)    

## Authentication

In [ ]:
from secrets import IDCLIENT, SECRET
auth_data['client_id'] = IDCLIENT
auth_data['client_secret'] = SECRET
# to get client_id and client_secret, create a web application (https://nanohub.org/developer/api/applications/new), use "https://127.0.0.1" as Redirect URL   

Login, CLogin = Auth.Login(
    Project,
    Component,
    client_id = auth_data['client_id'],
    client_secret = auth_data['client_secret'],
    url = "https://nanohub.org/api/developer/oauth/token",   
    open_state = STATE_LOGIN_OPEN
) 
Login.content.events["onAuth"] = [
    { "type": "stateChange", "modifies": STATE_LOGIN_OPEN, "newState": False},
    { "type": "propCall2", "calls": "buildSchema", "args": ['self'] }
]

In [ ]:
SETTINGS = {
    'values': {
        'type': 'String',
        'default_value': 'Animal Jumps Merrily',
        'label': 'text',
        'description': 'lowercase string that is going to be encoded'
    }, 'shift_input': {
        'type': 'Integer',
        'default_value': 24,
        'units': None,
        'min': 1,
        'max': 50,
        'label': 'Shifting',
        'description': 'integer that determines the shift amount for encoding'
    }, 'cores' : {
        "type": "Integer",
        "default_value": 1,
        "min" : 1, 
        "max" : 1,  
        "units" : "", 
        "description" : "Number of cores in in the venue",
        "label" : "Number of cores",
    }, "cutoff" : {
        "type": "Integer",
        "default_value": 0,
        "units" : "mins", 
        "min" : 0, 
        "max" : 0,     
        "description" : "Time cutoff",
        "label" : "Time cutoff",
    }, "venue" : {
        "type": "String",
        "default_value": "",
        "description" : "Venue identifier",
        "label" : "Venue identifier",
    }
}


## Defining Layout

In [ ]:
LAYOUT = {
    'input': {
        'type': 'group',
        'id': '',
        'label': 'Inputs',
        'enable': None,
        'layout': 'vertical',
        'children': [
            {
                'type': 'text',
                'id': 'values',
                'enable': None
            },{
                'type': 'number',
                'id': 'shift_input',
                'enable': None
            }
        ]
   }          
}


## Creating the settings panel

In [ ]:
url_sim = "https://nanohub.org/api/dbexplorer/simtools"
PNTOYSettings = AppBuilder.Settings(
    Project,
    Component, 
    SETTINGS,
    url=url_sim,
    toolname = TOOLNAME,
    revision = REVISION,
    layout = LAYOUT['input'],
    outputs = ['Cipher', 'Repeated Chars', 'Repeated Chars Count'],
    runSimulation = "simtool"
)
PNTOYSettings.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False},
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN, "newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS, "newState": '$e'}
]
PNTOYSettings.content.events["click"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True}
]

PNTOYSettings.content.events["submit"] = [
    { "type": "stateChange", "modifies": "parameters","newState": '$e.target.value'}
]

PNTOYSettings.content.events["onStatusChange"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_STATUS,"newState": "$e.target.value"}
]


eol = "\n"
js = ""
js += " (self, hashkey) => {" + eol
js += "  CacheStore.getItem(hashkey).then((value)=>{" + eol
js += "    var jsonOutput = JSON.parse(value);" + eol
js += "    var message = 'Cypher phrase : ' + jsonOutput['Cipher'] + '\\n'" + eol
js += "    message += 'Repeated Chars : ' + jsonOutput['Repeated Chars'] + '\\n'" + eol
js += "    self.setState({'alert_status':message});" + eol
js += "  });" + eol
js += "}" + eol
Component.addPropVariable("loadBaseOutput", {"type":"func", "defaultValue": js})


PNTOYSettings.content.events["onSuccess"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False },  
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN, "newState": False},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS, "newState": ''},
    { 
        "type": "stateChange", 
        "modifies": STATE_ALERT_OPEN, 
        "newState": True,
        "callbacks" : [
          {
            "type": "propCall2",
            "calls": "loadBaseOutput",
            "args": ['self', 'arguments[1]']
          }
        ] 
    }
]



## Creating Colorschema and adding nanoHUB logo

In [ ]:
ThemeProvider = MaterialBuilder.ThemeProvider( Component, MaterialBuilder.DefaultTheme(
    primary_color = '#699FBB',
    secondary_color = '#f1f1f1',
    primary_bg = '#FFFFFF',
    secondary_bg = '#dbeaf0',
    default_button = 'rgba(255, 255, 255, 0.87)',
    primary_button = 'rgba(255, 255, 255, 0.87)',
    secondary_button = 'rgba(0, 0, 0, 0.87)',
    default_button_bg = 'rgb(63, 162, 192)',
    primary_button_bg = 'rgba(0, 0, 0, 0.65)',
    secondary_button_bg = 'rgba(0, 0, 0, 0.12)',
))
AppBar = MaterialBuilder.AppBar(
    title="Caesar Cipher Tool"
)
logo = t.TeleportElement(t.TeleportContent(elementType="img"))
logo.content.attrs["width"] = "120"
logo.content.attrs["src"] = "https://nanohub.org/app/site/media/images/PressKit/nanoHUB_logo_color.jpg"
AppBar.content.children[0].addContent(logo)


## Assembling the app

In [ ]:
Gridv = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridv.content.attrs["container"] = True
Gridv.content.attrs["direction"] = "column"
Gridv.addContent(AppBar)
Gridv.addContent(PNTOYSettings)

ThemeProvider.addContent(Gridv)
ThemeProvider.addContent(Loader)
ThemeProvider.addContent(ErrorMessage)
ThemeProvider.addContent(AlertMessage)
ThemeProvider.addContent(Login)

Component.addNode(ThemeProvider)
Project.buildReact( TOOLNAME+"PROD"+".html");

## Displaying the app in jupyter

In [ ]:
from IPython.core.display import display
from IPython.display import IFrame
display(IFrame(src=TOOLNAME+"PROD"+".html", width="100%", height="400"))